Looking at the sentiment values for every james bond movie. OH baby.

Checks overall sentiment over time in each movie holy shit great idea.

Set srt file to dataframe, analyse dataframe.


In [20]:
# import subprocess

# subprocess.run(["subliminal", "download","-l ", "en", "Dr_No.mp4"])

In [21]:
import re 
import os
import sys
import glob
import time
import selenium
import numpy as np
import pandas as pd
from PIL import Image

from babelfish import *
from subliminal import *

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

from collections import Counter
from wordcloud import WordCloud, STOPWORDS

In [3]:
# get bond film table
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_James_Bond_films')
# get movie titles
jb_movies = df[0][2:-2][0].values
jb_flicks = []

for j in jb_movies:
    j = j.replace("[N]", '')
#     video = Video.fromname(f'{j}')
#     subtitle = list_subtitles([video], {Language('eng')})
#     subtitle = subtitle[video][0]
#     download_subtitles([subtitle])
#     save_subtitles(video, [subtitle], directory='Subtitles')
    pass

In [5]:
df_titles = df[0][0:1].values
df_main = pd.DataFrame(df[0][2:-2])
df_main.columns = df_titles[0]

# clean columns
df_main['Title'] = df_main.Title.apply(lambda x: x.replace('[N]', ''))
df_main.drop(df_main.index[4], inplace=True)

# Sort by decade
df_main['Decade'] = df_main['Year'].apply(lambda x: x[2]+'0')

df_main.reset_index(drop=True, inplace=True)
df_main.head(3)

,Title,Year,Bond actor,Director,Box office,Budget,Salary of Bond actor,Box office,Budget,Salary of Bond actor,Decade
0,Dr. No,1962,Sean Connery,Terence Young,59.5,1.1,0.1[b],448.8,7.0,0.6,60
1,From Russia with Love,1963,Sean Connery,Terence Young,78.9,2.0,0.3,543.8,12.6,1.6,60
2,Goldfinger,1964,Sean Connery,Guy Hamilton,124.9,3.0,0.5,820.4,18.6,3.2,60


In [ ]:
temp_srts = []
for movie in df1[0]:
    #get srt file
    temp_df = pd.

In [44]:
# the following check and clean srt specific lines
def is_time_stamp(l):
    if l[:2].isnumeric() and l[2] == ':':
        return True
    return False

def get_start_time(line):
    time = line.split()[0]
    print(time)
    if time[:2].isnumeric() and time[2] == ':':
        print(time)


def has_letters(line):
    if re.search('[a-zA-Z]', line):
        return True
    return False


def has_no_text(line):
    l = line.strip()
    if not len(l):
        return True
    if l.isnumeric():
        return True
    if is_time_stamp(l):
        return True
    if l[0] == '(' and l[-1] == ')':
        return True
    if not has_letters(line):
        return True
    return False


def is_lowercase_letter_or_comma(letter):
    if letter.isalpha() and letter.lower() == letter:
        return True
    if letter == ',':
        return True
    return False

# removes non-text lines, combines broken lines
def clean_up(lines):
    # add 'text' to list so the dataframe has a column name
    new_lines = []
    time_stamps = []
    temp_lst = []
    for line in lines[1:]:
        if has_no_text(line):
            if line[:2].isnumeric() and line[2] == ':':
                time_stamps.append(line[:8])
                continue
            continue
        elif len(new_lines) and is_lowercase_letter_or_comma(line[0]):
            #combine with previous line
            new_lines[-1] = new_lines[-1].strip() + ' ' + line
        else:
            #append line
            new_lines.append(line)
    for text, time in zip(new_lines, time_stamps):
        temp_line = [time, text.replace('\n', '').replace('-', '')]
        temp_lst.append(temp_line)
    return temp_lst


# convert srt to txt file from directory returns dict
def srt_to_txt(srt_file, encoding):
    with open(srt_file, encoding=encoding, errors='replace') as f:
        lines = f.readlines()
        new_lines = clean_up(lines)
    new_file_name = srt_file[:-4] + '.txt'
#     with open(new_file_name, 'w') as f:
#         # add 'text' to list so the dataframe has a column name
#         f.write('text\n')
#         for line in new_lines:
#             f.write(line)
    # for variable use in the main function and not to write to disk again
    return new_lines

# puts .txt file into a dataframe, returns df
def to_df(text):
    df = pd.read_csv(pd.compat.StringIO("\n".join(text)), sep=";")
    #remove puncutation
    df['text'] = df['text'].str.replace('[^\w\s]', '')
    return df

In [49]:
srt_test = 'Subtitles/GoldenEye.en.srt'
encoding = 'utf-8' 
temp_srt = srt_to_txt(srt_test, encoding)[1:] #ignore first advertisement 
temp_srt

[['00:02:58', 'I beg your pardon.'],
 ['00:03:46', 'I forgot to knock.'],
 ['00:03:51', "I'm alone."],
 ['00:03:54', "Aren't we all?"],
 ['00:03:57', " You're late, 007."],
 ['00:03:59', ' I had to stop in the bathroom.'],
 ['00:04:05', 'Ready to save the world again?'],
 ['00:04:09', 'After you, 006.'],
 ['00:04:52', 'James, for England.'],
 ['00:04:54', 'For England, Alec.'],
 ['00:04:57', "It's too easy."],
 ['00:05:00', 'Half of everything is luck, James.'],
 ['00:05:03', 'And the other half?'],
 ['00:05:32', 'Fate.'],
 ['00:05:36', ' Set timers: six minutes.'],
 ['00:05:39', ' Six minutes, check.'],
 ['00:05:56', 'Fire!'],
 ['00:06:00', 'Closing time, James!'],
 ['00:06:05', 'Last call!'],
 ['00:06:10', 'Buy me a pint.'],
 ['00:06:24',
  'This is Colonel Ourumov! Come out with your hands above your heads!'],
 ['00:06:28', 'How original.'],
 ['00:06:30', 'Shut the door, Alec!'],
 ['00:06:34', "There's a draft!"],
 ['00:06:36', 'Alec?'],
 ['00:06:39', 'Move out. Throw down your weap

In [50]:
va = SentimentIntensityAnalyzer()
def get_sentiment(tweet, adjust_sent=False):
    """
    VADER Analysis --> returns negative, neutral, positive, compounded sentiment.
    compound = normalized, weighted composite score between -1, 1

    :adjust_sent: Toggles if emojis will be replaced with subjective sentiment emojis
    """
    if adjust_sent:
        tweet = split_emojis(tweet, adjust_sent)
    score = va.polarity_scores(tweet)
    temp_list = [score['compound'], score['pos'], score['neu'], score['neg']]
    return temp_list

df_g = pd.DataFrame(temp_srt, columns=['Timestamp', 'Dialogue'])
df_g.head(3)

,Timestamp,Dialogue
0,00:02:58,I beg your pardon.
1,00:03:46,I forgot to knock.
2,00:03:51,I'm alone.
